In [9]:
import torch
import lfcm
from collections import OrderedDict
import numpy as np


In [10]:
model = lfcm.OldModel(gpu=0)


Inception v3 pretraining: Hardcoded strict=False, Might be omiting layers


In [11]:
checkpoint = torch.load('checkpoints/fcm_e4b16.pth')

In [12]:
model = lfcm.OldModel()
model = torch.nn.DataParallel(model, device_ids=[0]).cuda(0)
model.load_state_dict(checkpoint)

# new_state_dict = OrderedDict()
# for k, v in checkpoint.items():
#     name = k[7:] # remove module.
#     new_state_dict[name] = v

Inception v3 pretraining: Hardcoded strict=False, Might be omiting layers


<All keys matched successfully>

In [6]:
# model.load_state_dict(new_state_dict)


<All keys matched successfully>

In [14]:
empty_image = np.zeros((3, 299, 299), dtype=np.float32)
# image = torch.from_numpy(np.array(empty_image).copy())
image = torch.zeros(3, 299, 299)

empty_image_text = np.zeros(150, dtype=np.float32)
image_text = torch.from_numpy(empty_image_text.copy())

empty_text = np.zeros(150, dtype=np.float32)
text = torch.from_numpy(empty_text.copy())

with torch.no_grad():
    model.eval()
    output = model(image.unsqueeze(0), image_text.unsqueeze(0), text.unsqueeze(0))

    print(output)


tensor([[-0.3967,  0.1291]], device='cuda:0')


In [182]:
state_dict = torch.load('fcm_test.pth', map_location={'cuda:1':'cuda:0', 'cuda:2':'cuda:0', 'cuda:3':'cuda:0'})

In [189]:
model = lfcm.OldModel()
model = torch.nn.DataParallel(model, device_ids=[0]).cuda(0)
model.load_state_dict(state_dict)


Inception v3 pretraining: Hardcoded strict=False, Might be omiting layers


<All keys matched successfully>

In [184]:
import numpy as np
import customDataset
from dotenv import load_dotenv
import os
import torch
import random
from torch.utils.data import DataLoader
import lfcm
import torch.nn as nn 
from pylab import zeros
import torch.backends.cudnn as cudnn
import trainingFunctions as t


load_dotenv()


root_dir = os.getenv('ROOT_PATH')
split_train = 'train.csv'
split_val = 'valid.csv'
batch_size = 16
workers = 4
gpu = 0
gpus = [0]
weights = [0.2797, 1.0] 
class_weights = torch.FloatTensor(weights).cuda()
optimizer_name = 'ADAM'
epochs = 1
start_epoch = 0
ImgSize = 299
print_freq = 25

In [186]:
test_dataset = customDataset.CustomDatasetFCM(
    root_dir, split_val, Rescale=ImgSize,RandomCrop=0,Mirror=False, embed_dir='valid_embeddings')

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)

In [188]:
with torch.no_grad():
    model.eval()
    for i,(image, image_text, tweet, target) in enumerate(test_loader):

        image_var = torch.autograd.Variable(image)
        image_text_var = torch.autograd.Variable(image_text)
        tweet_var = torch.autograd.Variable(tweet)

        outputs = model(image_var, image_text_var, tweet_var)

        for idx,el in enumerate(outputs):
            topic_probs_str = ''
            for t in el:
                topic_probs_str = topic_probs_str + ',' + str(float(t))
            print(topic_probs_str)

        print(str(i) + ' / ' + str(len(test_loader)))

,-0.17930254340171814,0.2271994650363922
,-0.17930254340171814,0.2271994650363922
,-0.042774002999067307,0.1444185972213745
,0.07439956068992615,0.002719164825975895
,0.0476221889257431,0.1212974339723587
,-0.17930254340171814,0.2271994650363922
,0.057819850742816925,0.1316051036119461
,-0.17930254340171814,0.2271994650363922
,0.08749069273471832,0.09232635051012039
,-0.17930254340171814,0.2271994650363922
,-0.17930254340171814,0.2271994650363922
,-0.17930254340171814,0.2271994650363922
,-0.17930254340171814,0.2271994650363922
,-0.022609641775488853,0.07059711962938309
,-0.17930254340171814,0.2271994650363922
,-0.17930254340171814,0.2271994650363922
0 / 48
,-0.043442029505968094,0.12155210971832275
,-0.013975521549582481,0.05797142907977104
,-0.037794217467308044,0.1843976080417633
,0.07164429128170013,0.07409047335386276
,-0.17930254340171814,0.2271994650363922
,0.012230494059622288,0.023202722892165184
,-0.088339664041996,0.11510985344648361
,-0.17930254340171814,0.2271994650363922
,

In [190]:
scores = [-0.1794,  0.2271]
rscore = scores[1]
nrscore = scores[0]

softmax_hate_score = np.exp(rscore) / (np.exp(rscore) + np.exp(nrscore))

print(softmax_hate_score)

0.6002483483305081


In [191]:
np.arange(0, 1, 0.001)

array([0.   , 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008,
       0.009, 0.01 , 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017,
       0.018, 0.019, 0.02 , 0.021, 0.022, 0.023, 0.024, 0.025, 0.026,
       0.027, 0.028, 0.029, 0.03 , 0.031, 0.032, 0.033, 0.034, 0.035,
       0.036, 0.037, 0.038, 0.039, 0.04 , 0.041, 0.042, 0.043, 0.044,
       0.045, 0.046, 0.047, 0.048, 0.049, 0.05 , 0.051, 0.052, 0.053,
       0.054, 0.055, 0.056, 0.057, 0.058, 0.059, 0.06 , 0.061, 0.062,
       0.063, 0.064, 0.065, 0.066, 0.067, 0.068, 0.069, 0.07 , 0.071,
       0.072, 0.073, 0.074, 0.075, 0.076, 0.077, 0.078, 0.079, 0.08 ,
       0.081, 0.082, 0.083, 0.084, 0.085, 0.086, 0.087, 0.088, 0.089,
       0.09 , 0.091, 0.092, 0.093, 0.094, 0.095, 0.096, 0.097, 0.098,
       0.099, 0.1  , 0.101, 0.102, 0.103, 0.104, 0.105, 0.106, 0.107,
       0.108, 0.109, 0.11 , 0.111, 0.112, 0.113, 0.114, 0.115, 0.116,
       0.117, 0.118, 0.119, 0.12 , 0.121, 0.122, 0.123, 0.124, 0.125,
       0.126, 0.127,

In [194]:
np.arange(0.4, 0.44, 0.0000001)

array([0.4      , 0.4000001, 0.4000002, ..., 0.4399997, 0.4399998,
       0.4399999])